![img](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png) Filippo Miatto (2020) 

# 1. Derivatives and Optimization

Before we look into Automatic Differentiation (AD), we need to first understand why differentiation is useful for optimization. The idea is that differentiation gives us the ability to move the parameters of the simulation in the correct direction to increase or decrease the output of a given cost function. In order to apply differentiation to an algorithm, we need to know how to apply the __chain rule__ and to understand the computational types that are involved in differentiating functions and algorithms.

## 1.1 The chain rule
When we have concatenated functions such as $h(g(f(x))) = r$ and we want to compute the derivative of $r$ with respect to $x$, we need the chain rule. This rule can be visualized by imagining that the functions implement some sort of algorithm, and therefore that there is a of "data" through the functions: 

$$
x \xrightarrow{f} y \xrightarrow{g} z \xrightarrow{h} r
$$

The rule then says that if we want to write the derivative of $r$ with respect to $x$, we need to go backwards, through all of the the intermediate steps between $r$ and $x$:

$$
\frac{\partial r}{\partial x} = \frac{\partial r}{\partial z}\frac{\partial z}{\partial y}\frac{\partial y}{\partial x}
$$

If we have functions of multiple variables, we just sum over all the paths that take us from $r$ to $x$: for instance, if we have $h(y, z) = r$ where $y=f(x)$ and $z=g(x)$, then we have two paths:

$\require{AMScd}$
\begin{CD}
x @>f>> y\\
@VVgV        @VVhV\\
z @>h>> r
\end{CD}

and so in order to backpropagate the derivative of $r$ all the way to $x$ we must follow both branches:

$$
\frac{\partial r}{\partial x} = \frac{\partial r}{\partial z}\frac{\partial z}{\partial x} + \frac{\partial r}{\partial y}\frac{\partial y}{\partial x}
$$



#### Activiy 1: product rule (10 minutes)
Use the chain rule to prove $\frac{\partial}{\partial x}f(x)g(x) = f'(x)g(x) + f(x)g'(x)$

## 1.2 Types
In this section we will indicate the process of differentiation by $D$, so that we avoid specifying the name of the variables. Just remember that $D$ differentiates a function with respect to all of its inputs.

We gain a huge insight on differentiation by considering the _types_ that are involved. Recall that the type of a function that maps a type $a$ to a type $b$ is $(a\rightarrow b)$. Then, we can make the following argument.
If we have a function $f$ of type $(a\rightarrow b)$, the expression $D f$ represents a new function that takes a value $x\in a$ (in the domain of $f$) and it returns the _linear approximation_ of $f$ around $x$, which is also a function from $a$ to $b$, and it has type $(a \multimap b)$ (the symbol $\multimap$ is like an arrow, but it specifies that the function is linear). Therefore $D f$ has type $(a\rightarrow (a\multimap b))$ which means that $D$ has type:

$$
D: (a\rightarrow b) \rightarrow (a\rightarrow(a\multimap b))
$$

Take a few minutes to stare at the type of $D$ and let it sink in. This is what differentiation is. 
It takes a function $f$ and it gives us a new function $f'$, which takes a a point $x$ in the domain of $f$ and it returns the linear approximation of $f$ around $x$.

Let's see an example. Let's take a function $f$ from a vector space $a$ of dimension $n$ to $\mathbb{R}$. Then $Df$ is a function of type $(a\rightarrow (a\multimap \mathbb{R}))$, so if we evaluate $Df(x)$ where $x\in a$ we obtain a linear function that maps from $a$ to $\mathbb{R}$, which approximates what $f$ would have done on inputs around $x$. But what is a linear function that maps from a vector space $a$ to $\mathbb{R}$? It's a $1\times n$ matrix. This is what we normally intend when we talk about the gradient vector: we say it's a vector (which is fine), but we often forget that it is actually a linear map. This is especially true when $a$ is $\mathbb{R}$: in that case we say that $Df(x)$ is a number (which we also write as $f'(x)$), but it is actually the linear function that multiplies by that number.

#### Activity 2: playing with types (10 minutes)
Compute the type of $D^2$, i.e. of the double differentiation. Does it now make sense that the Hessian of a function from $\mathbb{R}^n$ to $\mathbb{R}$ is a matrix of shape $n\times n$?

## 1.3 Gradient Descent

The gradient of a function $f$ of type $(a\rightarrow \mathbb{R})$ has a very precise geometrical interpretation: it's the vector in the domain that points in the direction of steepest ascent, i.e. if the inputs move in that direction the output of $f$ increases the quickest (not forever, it increases at least for a small step, or mathematically speaking for an infinitesimal step). This implies that if we move in the opposite direction, the output of $f$ decreases. This is how we maximize or minimize functions using Gradient Descent: we follow the gradient or the opposite of the gradient in small steps.

Let's say we want to minimize the value of a function $f$ of type $(a\rightarrow \mathbb{R})$. We start by evaluating $f$ at some point $x\in a$ and we obtain a value $f(x)$. If we then evaluate the gradient of $f$ at $x$, we obtain a vector $f'(x)$ (actually, we obtain the linear approximation of $f$ at $x$, expressed as a $1\times n$ matrix, but we said that it's okay to think of it as a vector). Then if $\epsilon$ is small enough, it holds that

$$f(x - \epsilon f'(x)) < f(x) < f(x + \epsilon f'(x))$$

If our goal is minimization, we update $x$ by subtracting $\epsilon f'(x)$. If our goal is maximization we update $x$ by summing $\epsilon f'(x)$. Each of these updates is one step of the Gradient Descent/Ascent algorithm. We now just have to keep repeating the process and we will eventually end up in a local minimum.

So in summary, this is the Gradient Descent algorithm:
1. $\epsilon \leftarrow \text{learning rate}$ 
2. $x \leftarrow \text{initialize x}$
3. Repeat until you're happy: $x \leftarrow (x - \epsilon f'(x)$)

Note that there exist algorithms that implement more sophisticated updates. Some of these remember the direction of previous updates and use those values to give the search some "momentum", others adapt the value of the learning rate $\epsilon$ from step to step, and so on. One of the most famous algorithms for gradient descent is [Adam](https://arxiv.org/abs/1412.6980). However, they are all variations on the most important rule: that the gradient is the direction of steepest ascent.

#### Activity 3: hardcoded Gradient Descent (15 minutes)
Consider the function $f(x) = (x-3)^2$. Compute its derivative by hand, and code a function to implement it. Then use it in a `for` loop with 1000 steps to find the minimum of $f$ via Gradient Descent. Initialize $x = 0.0$ and use a learning rate $\epsilon = 0.01$.